In [ ]:
#Importing necessary libraries

import numpy as np
import pandas as pd

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns
import squarify

warnings.filterwarnings("ignore")

In [ ]:
#Reading data from CSV in ISO format

data = pd.read_csv("data.csv", encoding="ISO-8859-1")

In [ ]:
#Checking the head of dataframe to see the headers and data 

data.head()

# Let's do some EDA..

In [ ]:
#Summary of data/overall numbers

print("Shape of the dataset: ", data.shape)
print("\nColumns present: ", list(data.columns))
print("\nNo. of invoices: ", data['InvoiceNo'].nunique())
print("\nNo. of Products: ", data['Description'].nunique())
print("\nMin. & Max. Invoice date: ", min(data['InvoiceDate']), ' & ', max(data['InvoiceDate']))
print("\nMin. & Max. Qty: ", min(data['Quantity']), ' & ', max(data['Quantity']))
print("\nMin. & Max. Unit Price: ", min(data['UnitPrice']), ' & ', max(data['UnitPrice']))
print("\nNo. of Customers: ", data['CustomerID'].nunique())
print("\nNo. of Countries: ", data['Country'].nunique())

Findings...
A Total of 25900 invoices/unique transcations available
Transactions are dated till Nov 2011
Negative qty and price indicates there are some "return" transactions in the dataset
There are 4K cutsomers spanning 38 countries.

In [ ]:
#Cehcking for any NULL values:

col = ['InvoiceNo', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

for i in col:
    val = data[i].isnull().values.any()
    if val == True:
        #print(i, ' has null values: ', val)
        print('# of transactions with NULL', i, ':', data[data[i].isnull() == True].shape[0])

In [ ]:
#Since customer behave differently in different geogrophical regions, let's look at the customer distribution
# or invoice distribution by country for top10 countries.

temp = data[['Country', 'CustomerID', 'InvoiceNo']].groupby(['Country']).agg({"CustomerID": "nunique", 
                                                                             "InvoiceNo": "nunique"}).reset_index().sort_values(by='CustomerID', ascending=False).head(10)
ax = plt.figure(figsize=(15, 5)).subplots(1,2)
ax=sns.barplot(x="Country", y="CustomerID", data=temp, ax=ax[0])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Country wise customer count")


ax=sns.barplot(x="Country", y="InvoiceNo", data=temp)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Country wise invoice count")

Most of the data is coming from "United Kingdom", therefore we'll be restricting our analysis to only this country

In [ ]:
#Top Customers based on # of Invoices and Total $ spent

data['AmountSpent'] = data['Quantity'] * data['UnitPrice']
temp1 = data[['CustomerID', 'InvoiceNo', 'AmountSpent']].groupby(['CustomerID']).agg({"InvoiceNo": "nunique", 
                                                                             "AmountSpent": "sum"}).reset_index().sort_values(by=['InvoiceNo'], ascending=False).head(10)
temp2 = data[['CustomerID', 'InvoiceNo', 'AmountSpent']].groupby(['CustomerID']).agg({"InvoiceNo": "nunique", 
                                                                             "AmountSpent": "sum"}).reset_index().sort_values(by=['AmountSpent'], ascending=False).head(10)

ax = plt.figure(figsize=(15, 5)).subplots(1,2)
ax=sns.barplot(x="CustomerID", y="InvoiceNo", data=temp1, ax=ax[0])
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Top Customer via frequency")


ax=sns.barplot(x="CustomerID", y="AmountSpent", data=temp2)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Top Customer via amount spent")

Top customer based on frequency of visits is ID = 14911

Top customer based on $ amount spent is ID = 14646

Top customer based on dollar amount has less than half foorprint compared to the most frequent customer

In [ ]:
#Top 10 product sold

temp1 = data[['Description', 'Quantity']].groupby(['Description']).agg({"Quantity": "sum"}).reset_index().sort_values(by=['Quantity'], ascending=False).head(10)

ax = plt.figure(figsize=(15, 5))
ax=sns.barplot(x="Description", y="Quantity", data=temp1)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_title("Top selling products")

Top product is "World War 2 Gliders Asstd Designs" with over 50K units sold

# Data Cleaning...

To clean the dataset, we will follow some basic business logics:

1. Restrict the dataset to "UnitedKingdom"
2. Remove return orders
3. Remove rows where customerID are NA
4. Restrict the data to one year to better understand RFM

In [ ]:
data_uk = data[data['Country'] == 'United Kingdom']
print('Shape of dataset after selecting country = United Kingdom ', data_uk.shape)

data_uk = data_uk[data_uk['Quantity'] > 0]
print('Shape of dataset after removing return orders ', data_uk.shape)

data_uk.dropna(subset=['CustomerID'],how='all',inplace=True, )
print('Shape of dataset after removing Null CustomerID ', data_uk.shape)


data_uk = data_uk[data_uk['InvoiceDate']>= "2010-12-09"]
print('Shape of dataset after removing more than 1 year worth of data ', data_uk.shape)

In [ ]:
#Summary of cleaned dataset

print("Shape of the dataset: ", data_uk.shape)
print("No. of invoices: ", data_uk['InvoiceNo'].nunique())
print("No. of Products: ", data_uk['Description'].nunique())
print("Min. & Max. Invoice date: ", min(data_uk['InvoiceDate']), ' & ', max(data_uk['InvoiceDate']))
print("Min. & Max. Qty: ", min(data_uk['Quantity']), ' & ', max(data_uk['Quantity']))
print("Min. & Max. Unit Price: ", min(data_uk['UnitPrice']), ' & ', max(data_uk['UnitPrice']))
print("No. of Customers: ", data_uk['CustomerID'].nunique())

# Making RFM dataframe

In [ ]:
#To start with the RFM, we need to create, R(Recency) and F(Frequency) and M(Monetary) columns

In [ ]:
#Recency Column

data_uk['Date'] = pd.DatetimeIndex(data_uk['InvoiceDate']).date


#We will make the last date as a reference
now = dt.date(2011,9,30)
print('Base or reference date: ', now)


#Calculate the last visit date
data_recency = data_uk.groupby(by='CustomerID', as_index=False)['Date'].max()
data_recency = data_recency.rename(columns={'CustomerID': 'CustomerID', 'Date': 'LastPurshaceDate'})
data_recency['Recency'] = data_recency['LastPurshaceDate'].apply(lambda x: (now - x).days)

#Drop LastPurchaseDate
data_recency.drop('LastPurshaceDate',axis=1,inplace=True)


data_recency.head()

In [ ]:
#Frequency Column

temp = data_uk
temp.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)

#calculate frequency of purchases
data_freq = temp.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
data_freq = data_freq.rename(columns={'CustomerID': 'CustomerID', 'InvoiceNo': 'Frequency'})


data_freq.head()

In [ ]:
#Monetary Column

data_monetary = data_uk.groupby(by=['CustomerID'], as_index=False)['AmountSpent'].sum()
data_monetary = data_monetary.rename(columns={'CustomerID': 'CustomerID', 'AmountSpent': 'Monetary'})


data_monetary.head()

In [ ]:
#Combining Recency, Frequency and Monetary dataframe


data_rfm = data_recency.merge(data_freq, how='left', on='CustomerID').merge(data_monetary, how='left', on='CustomerID')

In [ ]:
data_rfm.head()

# RFM analysis using Quantiles

Here we will create customers segments from RFM Model using Quartiles. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

In [ ]:
#Creating Quantiles dataframe for 25%, 50%, 75% to create 4 buckets

quantiles = data_rfm.drop('CustomerID', axis=1).quantile(q=[0.25,0.5,0.75])
quantiles

In [ ]:
#Function to create segments of Recency

def RScore(x, p, quantiles):
    #return quantiles[quantiles.index == 0.25][p]
    if x <= int(quantiles[quantiles.index == 0.25][p]):
        return 4
    elif x <= int(quantiles[quantiles.index == 0.50][p]):
        return 3
    elif x <= int(quantiles[quantiles.index == 0.75][p]): 
        return 2
    else:
        return 1

    
#Function to create segments of Frequency and Monetary
def FMScore(x, p, quantiles):
    if float(x) <= float(quantiles[quantiles.index == 0.25][p]):
        return 1
    elif float(x) <= float(quantiles[quantiles.index == 0.50][p]):
        return 2
    elif float(x) <= float(quantiles[quantiles.index == 0.75][p]): 
        return 3
    else:
        return 4

In [ ]:
#Calculating each customer's R/F/M quartiles based on quantiles table and functions defined above

rfm_segment = data_rfm

rfm_segment['R_Quartile'] = rfm_segment['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segment['F_Quartile'] = rfm_segment['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segment['M_Quartile'] = rfm_segment['Monetary'].apply(FMScore, args=('Monetary',quantiles,))



rfm_segment['RFMScore'] = rfm_segment.R_Quartile.map(str) +rfm_segment.F_Quartile.map(str) +rfm_segment.M_Quartile.map(str)
rfm_segment.head()

We can define various customer segments based on our RFM segment:

1.Core - Best Customers
 
 RFM Score: 444
 
 Who They Are: Highly engaged customers who visited recently, the most often visited and generates the most revenue.


2.Loyal - Most Loyal Customers
 
 RFM Score: X4X
 
 Who They Are: Customers who are most frequent
 
 
3.BigSpenders - Highest Paying Customers
 
 RFM Score: XX4
 
 Who They Are: Customers who have generated the most revenue
 
 
4.Rookies - Your Newest Customers
 
 RFM Score: 41X
 
 Who They Are: First time buyers


5.LoosingBigSpenders - Focust point!! where good customers are slipping

RFM Score: 244 and 234

Who They Are: Customers who were frequent and big spenders but have not visited the store in a while


6.LostBigSpenders - Good customers who are lost

RFM Score: 144 and 134

Who They Are: Customers who were frequent and big spenders but have not visited the store from a long time


7.LostCheapSpenders - Bad/Cheap customers who are lost

RFM Score: 141 and 131

Who They Are: Customers who were frequent and big spenders but have not visited the store from a long time


8.Regular - The customers having common behaviour across these metrics

RFM Score: Remaining Scores

Who They Are: Customer who have average metrics across each RFM scores

In [ ]:
#Snapshot of some of the KPI’s against each customer segment

KPI = pd.DataFrame(columns=['Segment', 'CustomerCount', 'RecencyMean', 'FrequencyMean', 
                            'MonetaryMean', '%Customer', '%Sales'])
KPI['Segment'] = ['Core', 'Loyal', 'BigSpenders', 'Rookies', 'LoosingBigSpenders', 'LostBigSpenders', 'LostCheapSpenders']


In [ ]:
# segment_dict = {'Core':[(4, 4, 4)], 
#                 'Loyal':[('X', 4, 'X')], 
#                 'BigSpenders':[('X', 'X', 4)], 
#                 'Rookies':[(4, 1, 'X')], 
#                 'LoosingBigSpenders':[(2, 4, 4), (2, 3, 4)], 
#                 'LostBigSpenders':[(1, 4, 4), (1, 3, 4)], 
#                 'LostCheapSpenders':[(1,4, 1), (1, 3, 1)]}

In [ ]:
#Segment == Core (444)

CustomerCount = rfm_segment[rfm_segment.RFMScore == '444']['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.RFMScore == '444']['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.RFMScore == '444']['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.RFMScore == '444']['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.RFMScore == '444']['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.RFMScore == '444']['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[0].CustomerCount = CustomerCount
KPI.loc[0].RecencyMean = RecencyMean
KPI.loc[0].FrequencyMean = FrequencyMean
KPI.loc[0].MonetaryMean = MonetaryMean
KPI.loc[0]['%Customer'] = PerCustomer
KPI.loc[0]['%Sales'] = PerSales

In [ ]:
# Segment == Loyal (X4X)

CustomerCount = rfm_segment[rfm_segment.F_Quartile == 4]['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.F_Quartile == 4]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.F_Quartile == 4]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.F_Quartile == 4]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.F_Quartile == 4]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.F_Quartile == 4]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[1].CustomerCount = CustomerCount
KPI.loc[1].RecencyMean = RecencyMean
KPI.loc[1].FrequencyMean = FrequencyMean
KPI.loc[1].MonetaryMean = MonetaryMean
KPI.loc[1]['%Customer'] = PerCustomer
KPI.loc[1]['%Sales'] = PerSales

In [ ]:
# Segment == BigSpenders (XX4)

CustomerCount = rfm_segment[rfm_segment.M_Quartile == 4]['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.M_Quartile == 4]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.M_Quartile == 4]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.M_Quartile == 4]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.M_Quartile == 4]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.M_Quartile == 4]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[2].CustomerCount = CustomerCount
KPI.loc[2].RecencyMean = RecencyMean
KPI.loc[2].FrequencyMean = FrequencyMean
KPI.loc[2].MonetaryMean = MonetaryMean
KPI.loc[2]['%Customer'] = PerCustomer
KPI.loc[2]['%Sales'] = PerSales

In [ ]:
# Segment == Rookies (41X)

CustomerCount = rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['CustomerID'].count()
RecencyMean = round(rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[(rfm_segment.R_Quartile == 4) & (rfm_segment.F_Quartile == 1)]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[3].CustomerCount = CustomerCount
KPI.loc[3].RecencyMean = RecencyMean
KPI.loc[3].FrequencyMean = FrequencyMean
KPI.loc[3].MonetaryMean = MonetaryMean
KPI.loc[3]['%Customer'] = PerCustomer
KPI.loc[3]['%Sales'] = PerSales

In [ ]:
# Segment == LoosingBigSpenders (244, 234)

CustomerCount = rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.RFMScore.isin(['244', '234'])]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[4].CustomerCount = CustomerCount
KPI.loc[4].RecencyMean = RecencyMean
KPI.loc[4].FrequencyMean = FrequencyMean
KPI.loc[4].MonetaryMean = MonetaryMean
KPI.loc[4]['%Customer'] = PerCustomer
KPI.loc[4]['%Sales'] = PerSales

In [ ]:
# Segment == LostBigSpenders (144, 134)

CustomerCount = rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.RFMScore.isin(['144', '134'])]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[5].CustomerCount = CustomerCount
KPI.loc[5].RecencyMean = RecencyMean
KPI.loc[5].FrequencyMean = FrequencyMean
KPI.loc[5].MonetaryMean = MonetaryMean
KPI.loc[5]['%Customer'] = PerCustomer
KPI.loc[5]['%Sales'] = PerSales

In [ ]:
# Segment == LostCheapSpenders (141, 131)

CustomerCount = rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['CustomerID'].count()
RecencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['Recency'].mean(), 2)
FrequencyMean = round(rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['Frequency'].mean(), 2)
MonetaryMean = round(rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['Monetary'].mean(), 2)
PerCustomer = round((rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['CustomerID'].count() /\
                     rfm_segment['CustomerID'].count())*100, 2)
PerSales = round((rfm_segment[rfm_segment.RFMScore.isin(['141', '131'])]['Monetary'].sum() /\
                     rfm_segment['Monetary'].sum())*100, 2)


KPI.loc[6].CustomerCount = CustomerCount
KPI.loc[6].RecencyMean = RecencyMean
KPI.loc[6].FrequencyMean = FrequencyMean
KPI.loc[6].MonetaryMean = MonetaryMean
KPI.loc[6]['%Customer'] = PerCustomer
KPI.loc[6]['%Sales'] = PerSales

In [ ]:
KPI

In [ ]:
#Visualizing the RFM segments

fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(15, 8)
squarify.plot(sizes=KPI['CustomerCount'], 
              label=['Core',
                     'Loyal',
                     'BigSpenders',
                     'Rookies',
                     'LoosingBigSpenders', 
                     'LostBigSpenders',
                     'LostCheapSpenders'], alpha=0.8, )
plt.title("RFM Segments",fontsize=15,fontweight="bold")
plt.axis('off')
plt.show()